# Check for ion leakage over time
To check whether ion leakage over time occurred, the tyrosine phospho ion intensity curve over subsequent higher-energy MS2 spectra was examined.

In [1]:
import pandas as pd
from pyopenms import MzMLFile, MSExperiment
import numpy as np
import re
import matplotlib.pyplot as plt

In [ ]:
tolerance_margin = 216.042 * 10 / 1e6
print(216.042 - tolerance_margin, 216.042 + tolerance_margin)

In [3]:
def ion_detection(spectrum):
    spectrum_mz, intensities = spectrum.get_peaks()
    lower_border = np.searchsorted(spectrum_mz, 216.042 - tolerance_margin, side="left")
    higher_border = np.searchsorted(spectrum_mz, 216.042 + tolerance_margin, side="right")

    detected_peaks_mz = spectrum_mz[lower_border:higher_border]
    detected_peaks_intensities = intensities[lower_border:higher_border]
    if len(detected_peaks_mz) == 0:
        return None, 0
    max_peak_idx = np.argmax(detected_peaks_intensities)
    max_peak_mz = detected_peaks_mz[max_peak_idx]
    max_peak_intensity = detected_peaks_intensities[max_peak_idx]
    return max_peak_mz, max_peak_intensity

In [4]:
exp = MSExperiment()

MzMLFile().load("../../data/230928_JL_Immonium_ions_Modified_DIA.mzML", exp)
spectra_all = exp.getSpectra()

## Distribution of peak intensity differences across consecutive spectra

In [5]:
def get_peaks_differences(intensities):
    differences = np.diff(intensities)
    # Leave out the last element because there is no corresponding diff
    # And only take into account peaks that are >= 10% of the mean intensity to remove noise
    peaks_mask = (intensities >= np.mean(intensities)/10)[:-1]
    return differences[peaks_mask]

In [39]:
def get_peaks_differences_for_whole_experiment():
    intensities_diffs = []
    spectrum_idx = 1

    while spectrum_idx < len(spectra_all):
        intensities_per_ms1_spectrum = []
        current_spectrum = spectra_all[spectrum_idx]

        while current_spectrum.getMSLevel() == 2:
            if current_spectrum.getPrecursors()[0].getMetaValue("collision energy") == 50:
                current_ms1_intensity = ion_detection(current_spectrum)[1]
                intensities_per_ms1_spectrum.append(current_ms1_intensity)

            spectrum_idx += 1
            if spectrum_idx == len(spectra_all):
                break
            current_spectrum = spectra_all[spectrum_idx]
        intensities_per_ms1_spectrum = np.array(intensities_per_ms1_spectrum)
        if not np.all(intensities_per_ms1_spectrum == 0):
            print(spectrum_idx)
            intensities_diffs.append(get_peaks_differences(intensities_per_ms1_spectrum))
        spectrum_idx += 1
    return intensities_diffs


In [ ]:
diffs_phospho = get_peaks_differences_for_whole_experiment()

In [ ]:
plt.hist(np.abs(np.concatenate(diffs_phospho)), bins=np.logspace(start=1, stop=8))
plt.xscale("log")
plt.xlabel("Absolute intensity difference")
plt.savefig("../../data/plots/ion_intensity_differences_distribution.png", bbox_inches="tight")

## Example intensity curves along the MS2 spectra within an MS1 window

In [8]:
def plot_phospho_ions_behaviour_for_spectrum_id_within_ms1_window(s_id_number):
    current_ms1_intensities = []
    previous_ms1_intensities = []
    spectrum_idcs = []
    
    initial_spectrum_idx = [i[0] for i in enumerate(spectra_all) if i[1].getNativeID() == f"controllerType=0 controllerNumber=1 scan={s_id_number}"][0]
    ms1_spectrum_idx = exp.getPrecursorSpectrum(initial_spectrum_idx)
    spectrum_idx = ms1_spectrum_idx + 1

    current_spectrum = spectra_all[spectrum_idx]
    while current_spectrum.getMSLevel() == 2:
        if current_spectrum.getPrecursors()[0].getMetaValue("collision energy") == 50:
            current_ms1_intensity = ion_detection(current_spectrum)[1]
            previous_ms1_intensity = ion_detection(spectra_all[spectrum_idx - 73])[1]
            current_ms1_intensities.append(current_ms1_intensity)
            previous_ms1_intensities.append(previous_ms1_intensity)
            spectrum_idcs.append(spectrum_idx)

        spectrum_idx += 1
        current_spectrum = spectra_all[spectrum_idx]

    plt.figure(figsize=(6, 4))
    plt.plot(spectrum_idcs, current_ms1_intensities, marker="o")
    plt.xlabel("Spectrum ID")
    plt.ylabel("Phospho ion intensity")
    plt.savefig(f"../../data/plots/ion_intensity_over_windows_example_{s_id_number}.png", bbox_inches="tight")

In [ ]:
plot_phospho_ions_behaviour_for_spectrum_id_within_ms1_window(24000)

In [ ]:
plot_phospho_ions_behaviour_for_spectrum_id_within_ms1_window(73043)